In [3]:
import torch

model_dir = "pretrained-Japanese-Extra"
step = 0

g_model = torch.load(f"{model_dir}/G_{step}.pth", map_location="cpu")
d_model = torch.load(f"{model_dir}/D_{step}.pth", map_location="cpu")
wd_model = torch.load(f"{model_dir}/WD_{step}.pth", map_location="cpu")

In [8]:
g_dict = {}
skip_list = ["enc_p.in_feature_net.", "enc_p.emo_vq.", "enc_p.out_feature_net."]
for key in g_model["model"].keys():
    if any([key.startswith(s) for s in skip_list]):
        print(f"skip {key}")
        continue
    else:
        g_dict[key] = g_model["model"][key]

skip enc_p.in_feature_net.0.weight
skip enc_p.in_feature_net.2.weight
skip enc_p.in_feature_net.2.bias
skip enc_p.in_feature_net.3.norm.weight
skip enc_p.in_feature_net.3.norm.bias
skip enc_p.in_feature_net.3.mlp.c_fc1.weight
skip enc_p.in_feature_net.3.mlp.c_fc2.weight
skip enc_p.in_feature_net.3.mlp.c_proj.weight
skip enc_p.in_feature_net.4.weight
skip enc_p.emo_vq._codebook.initted
skip enc_p.emo_vq._codebook.cluster_size
skip enc_p.emo_vq._codebook.embed_avg
skip enc_p.emo_vq._codebook.embed
skip enc_p.out_feature_net.weight
skip enc_p.out_feature_net.bias


In [9]:
d_dict = {}
for key in d_model["model"].keys():
    d_dict[key] = d_model["model"][key]

In [10]:
wd_dict = {}
for key in wd_model["model"].keys():
    wd_dict[key] = wd_model["model"][key]

In [11]:
from safetensors.torch import save_file


save_file(g_dict, f"G_0.safetensors")
save_file (d_dict, f"D_0.safetensors")
save_file (wd_dict, f"WD_0.safetensors")